<a href="https://colab.research.google.com/github/siumingdev/coursera-financial-engineering/blob/main/course_1_option_pricing_binomial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Option Pricing in the Multi-Period Binomial Model


In [4]:
import numpy as np

In [5]:
def format_float_array(arr: np.ndarray) -> str:
    return "[" + ", ".join(f"{a:.2f}" for a in arr) + "]"

In [17]:
def option_price(
    K: float,
    T: float,
    S_0: float,
    r: float,
    vol: float,
    c: float,
    is_put: bool,
    is_american: bool,
    use_future_price: bool,
    N: int,
    print_details: bool = False
) -> float:
    """
    Prices an American option using a binomial tree.

    Args:
        K: Strike price.
        T: Time to maturity.
        S_0: Initial underlying price.
        r: Risk-free interest rate.
        vol: Volatility.
        c: Dividend yield.
        is_put: True for a put option, False for a call.
        is_american: True for an American option, False for a European option.
        use_future_price: True to use the future price of the underlying as the initial price.
        N: Number of time steps.
        print_details: Whether to print intermediate calculations.

    Returns:
        The price of the American option.
    """

    delta_t = T / N
    u = np.exp(vol * np.sqrt(delta_t))
    d = 1 / u
    q = (np.exp((r - c) * delta_t) - d) / (u - d)
    R = np.exp(r * delta_t)

    def get_underlying_prices(i) -> np.ndarray:
        return S_0 * np.power(u, i - np.arange(i + 1)) * np.power(d, np.arange(i + 1))

    def get_payoffs(S: np.ndarray) -> np.ndarray:
        return np.maximum(K - S, 0) if is_put else np.maximum(S - K, 0)

    # Initialize underlying prices at maturity
    S = get_underlying_prices(N)

    # Initialize option values at maturity
    C = get_payoffs(S)

    if print_details:
        print(f"Stock Price at maturity: {format_float_array(S)}")
        print(f"Option Value at maturity: {format_float_array(C)}")

    # Work backwards through the tree
    for i in range(N - 1, -1, -1):
        C_no_early_ex = (q * C[:-1] + (1 - q) * C[1:]) / R
        if is_american:
            S = (q * S[:-1] + (1 - q) * S[1:]) if use_future_price else get_underlying_prices(i)
            payoffs = get_payoffs(S)
            should_early_ex = payoffs > C_no_early_ex
            C = np.where(should_early_ex, payoffs, C_no_early_ex)
        else:
            C = C_no_early_ex

        if print_details:
            print(f"Step: {i}")
            print(f"Stock Price: {format_float_array(S)}")
            print(f"Option Value: {format_float_array(C)}")
            if is_american:
                print(f"Option Value (no early exercise): {format_float_array(C_no_early_ex)}")
                print(f"Early Exercise: {should_early_ex}")

    return C[0]


In [18]:
K = 110
T = 0.25
S_0 = 100
r = 0.02
vol = 0.3
c = 0.01

In [19]:
# Q1
call_price = option_price(
    K=K,
    T=T,
    S_0=S_0,
    r=r,
    vol=vol,
    c=c,
    is_put=False,
    is_american=True,
    use_future_price=False,
    N=15,
    print_details=True
)

Stock Price at maturity: [178.77, 165.45, 153.12, 141.70, 131.14, 121.37, 112.32, 103.95, 96.20, 89.03, 82.39, 76.25, 70.57, 65.31, 60.44, 55.94]
Option Value at maturity: [68.77, 55.45, 43.12, 31.70, 21.14, 11.37, 2.32, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Step: 14
Stock Price: [171.98, 159.16, 147.30, 136.32, 126.16, 116.76, 108.05, 100.00, 92.55, 85.65, 79.26, 73.36, 67.89, 62.83, 58.15]
Option Value: [61.99, 49.17, 37.31, 26.33, 16.18, 6.77, 1.14, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Option Value (no early exercise): [61.99, 49.17, 37.31, 26.33, 16.18, 6.77, 1.14, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Early Exercise: [False False False False False False False False False False False False
 False False False]
Step: 13
Stock Price: [165.45, 153.12, 141.70, 131.14, 121.37, 112.32, 103.95, 96.20, 89.03, 82.39, 76.25, 70.57, 65.31, 60.44]
Option Value: [55.47, 43.14, 31.73, 21.17, 11.40, 3.91, 0.56, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Option V

In [20]:
# Q2
put_price = option_price(
    K=K,
    T=T,
    S_0=S_0,
    r=r,
    vol=vol,
    c=c,
    is_put=True,
    is_american=True,
    use_future_price=False,
    N=15,
    print_details=True
)

Stock Price at maturity: [178.77, 165.45, 153.12, 141.70, 131.14, 121.37, 112.32, 103.95, 96.20, 89.03, 82.39, 76.25, 70.57, 65.31, 60.44, 55.94]
Option Value at maturity: [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 6.05, 13.80, 20.97, 27.61, 33.75, 39.43, 44.69, 49.56, 54.06]
Step: 14
Stock Price: [171.98, 159.16, 147.30, 136.32, 126.16, 116.76, 108.05, 100.00, 92.55, 85.65, 79.26, 73.36, 67.89, 62.83, 58.15]
Option Value: [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 3.07, 10.00, 17.45, 24.35, 30.74, 36.64, 42.11, 47.17, 51.85]
Option Value (no early exercise): [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 3.07, 9.98, 17.43, 24.33, 30.71, 36.62, 42.09, 47.15, 51.83]
Early Exercise: [False False False False False False False  True  True  True  True  True
  True  True  True]
Step: 13
Stock Price: [165.45, 153.12, 141.70, 131.14, 121.37, 112.32, 103.95, 96.20, 89.03, 82.39, 76.25, 70.57, 65.31, 60.44]
Option Value: [0.00, 0.00, 0.00, 0.00, 0.00, 1.56, 6.59, 13.80, 20.97, 27.61, 33.75, 39.43, 44.69, 49.56]

In [21]:
# Q5
# check put call parity
# P + S exp(-cT) - C - K exp(-rT) should be 0

put_price + S_0 * np.exp(-c * T) - call_price - K * np.exp(-r * T)

0.05464719286931086

In [22]:
def price_option_on_future(
    K: float,
    T: float,
    S_0: float,
    r: float,
    vol: float,
    c: float,
    is_put: bool,
    is_american: bool,
    N: int,
    N_option: int,
    print_details: bool = False
) -> float:
    """
    Prices an American option using a binomial tree.

    Args:
        K: Strike price.
        T: Time to maturity of the underlying future.
        S_0: Initial underlying price.
        r: Risk-free interest rate.
        vol: Volatility.
        c: Dividend yield.
        is_put: True for a put option, False for a call.
        is_american: True for an American option, False for a European option.
        N: Number of time steps.
        N_option: Number of time steps of the option maturity.
        print_details: Whether to print intermediate calculations.

    Returns:
        The price of the American option.
    """

    assert N_option <= N

    delta_t = T / N
    u = np.exp(vol * np.sqrt(delta_t))
    d = 1 / u
    q = (np.exp((r - c) * delta_t) - d) / (u - d)
    R = np.exp(r * delta_t)

    def get_underlying_prices(i) -> np.ndarray:
        return S_0 * np.power(u, i - np.arange(i + 1)) * np.power(d, np.arange(i + 1))

    def get_payoffs(S: np.ndarray) -> np.ndarray:
        return np.maximum(K - S, 0) if is_put else np.maximum(S - K, 0)

    S = get_underlying_prices(N)
    for i in range(N - 1, N_option - 1, -1):
        S = (q * S[:-1] + (1 - q) * S[1:])

    C = get_payoffs(S)

    if print_details:
        print(f"Stock Price at maturity: {format_float_array(S)}")
        print(f"Option Value at maturity: {format_float_array(C)}")

    for i in range(N_option - 1, -1, -1):
        C_no_early_ex = (q * C[:-1] + (1 - q) * C[1:]) / R
        if is_american:
            S = (q * S[:-1] + (1 - q) * S[1:])
            payoffs = get_payoffs(S)
            should_early_ex = payoffs > C_no_early_ex
            C = np.where(should_early_ex, payoffs, C_no_early_ex)
        else:
            C = C_no_early_ex

        if print_details:
            print(f"Step: {i}")
            print(f"Stock Price: {format_float_array(S)}")
            print(f"Option Value: {format_float_array(C)}")
            if is_american:
                print(f"Option Value (no early exercise): {format_float_array(C_no_early_ex)}")
                print(f"Early Exercise: {should_early_ex}")

    return C[0]


In [23]:
# Q6

price_option_on_future(
    K=K,
    T=T,
    S_0=S_0,
    r=r,
    vol=vol,
    c=c,
    is_put=False,
    is_american=True,
    N=15,
    N_option=10,
    print_details=True
)

Stock Price at maturity: [147.42, 136.43, 126.26, 116.85, 108.14, 100.08, 92.62, 85.72, 79.33, 73.42, 67.95]
Option Value at maturity: [37.42, 26.43, 16.26, 6.85, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Step: 9
Stock Price: [141.85, 131.27, 121.49, 112.43, 104.05, 96.30, 89.12, 82.48, 76.33, 70.64]
Option Value: [31.85, 21.27, 11.49, 3.37, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Option Value (no early exercise): [31.83, 21.27, 11.48, 3.37, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
Early Exercise: [ True  True  True False False False False False False False]
Step: 8
Stock Price: [136.48, 126.31, 116.89, 108.18, 100.12, 92.65, 85.75, 79.36, 73.44]
Option Value: [26.48, 16.31, 7.37, 1.66, 0.00, 0.00, 0.00, 0.00, 0.00]
Option Value (no early exercise): [26.47, 16.30, 7.37, 1.66, 0.00, 0.00, 0.00, 0.00, 0.00]
Early Exercise: [ True  True False False False False False False False]
Step: 7
Stock Price: [131.32, 121.53, 112.47, 104.09, 96.33, 89.15, 82.50, 76.36]
Option Value: [21.32, 11.77, 4.47, 0.8

1.662672607788038